![Rampart Logo](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/logo.png?raw=1)

Auge is an image classification model. Its main target's to recognize common photos to determine a few flat publication's features. Each image belongs to a specific realty, having a bunch of photo & panorama recognized **twinkle** can better predict apartments' order.

## I/O
Required images lie at `/content/drive/MyDrive/Scientific/v3/Images/` .

Currently the DB containes images of two types: simple photos & wide (360 deg) panoramas. Until a stable classification model panoramas should be omitted. Final classifier must be stored into `/content/drive/MyDrive/Scientific/v3/Models/auge.pth` .

## Classes
- `luxury` is a flat with rich furniture, huge rooms, chandeliers, fireplaces, etc.

![Luxury 1](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/luxury1.webp?raw=1)
![Luxury 2](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/luxury2.webp?raw=1)
![Luxury 3](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/luxury3.webp?raw=1)

- `comfort` is the most suitable for an ordinary citizen apartments. Clean, neat, sometimes minimalistic, average area, qualitive furniture, etc.

![Comfort 1](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/comfort1.webp?raw=1)
![Comfort 2](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/comfort2.webp?raw=1)
![Comfort 3](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/comfort3.webp?raw=1)

- `construction` is a flat without a finished design. No doors, floor, supplies, wallpapers, ceiling, furniture, etc. Typically, new buildings contain these apartments.

![Construction 1](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/construction1.webp?raw=1)
![Construction 2](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/construction2.webp?raw=1)
![Construction 3](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/construction3.webp?raw=1)

- `junk` is an old flat image. Probably, the whole apartments should belong to a dormitory, Khrushchevka or gostinka.

![Junk 1](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/junk1.webp?raw=1)
![Junk 2](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/junk2.webp?raw=1)
![Junk 3](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/junk3.webp?raw=1)

- `excess` is the trash category. Actually, all exterior photos, outlines & posters lie here.

![Excess 1](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/excess1.webp?raw=1)
![Excess 2](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/excess2.webp?raw=1)
![Excess 3](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/excess3.webp?raw=1)

In [ ]:
%matplotlib inline

In [ ]:
from plotly.graph_objs import Pie, Figure, Scatter
from plotly.figure_factory import create_annotated_heatmap
from re import match
from numpy import arange, trace, sum
from glob import glob
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torch.nn import Module, Conv2d, Linear, ReLU, CrossEntropyLoss, Sequential
from torch.optim import Adam
from torch import no_grad, save, max, load, zeros, cat, long, device
from torch.cuda import is_available
from PIL.Image import open
from time import time
from google.colab.drive import mount, flush_and_unmount
from uuid import uuid4

In [ ]:
mount('/content/drive')

In [ ]:
labels = ['luxury', 'comfort', 'junk', 'construction', 'excess']
mappings = {l: i for i, l in enumerate(labels)}

In [ ]:
def parse(path):
    result = match(r'^.*/(\w+)\.(\w+)\.webp$', path)
    return ('unknown', 'unknown') if not result else result.groups()    

In [ ]:
images = DataFrame({'path': glob('/content/drive/MyDrive/Scientific/V3/Images/*.webp')})
images['label'] = images['path'].apply(lambda p: mappings[parse(p)[1]])

In [ ]:
counts = images['label'].value_counts().sort_index()
figure = Figure()
figure.add_trace(Pie(labels=[labels[i] for i in counts.index], values=counts.values))
figure.update_layout(margin={'t': 30, 'r': 0, 'b': 0, 'l': 0})
figure.show(renderer='colab')

In [ ]:
class AugeDataset(Dataset):
    def __init__(self, data):
        self._data = data.values
        self._transforms = Compose(
            [
                ToTensor(),
                Resize((230, 310))
            ]
        )

    def __len__(self):
        return len(self._data)

    def __getitem__(self, index):
        path, label = self._data[index]
        return self._transforms(open(path)), label

In [ ]:
training_images, testing_images = train_test_split(images, test_size=0.3, stratify=images['label'])
training_images, validation_images = train_test_split(
    training_images,
    test_size=0.3,
    stratify=training_images['label']
)
batch_size = 4
training_loader = DataLoader(AugeDataset(training_images), batch_size, True)
validation_loader = DataLoader(AugeDataset(validation_images), batch_size)
testing_loader = DataLoader(AugeDataset(testing_images), batch_size)

In [ ]:
class View(Module):
    def __init__(self, *shape):
        super().__init__()
        self._shape = shape

    def forward(self, x):
        return x.view(*self._shape)

In [ ]:
class Auge(Module):
    def __init__(self):
        super().__init__()
        self._sequential = Sequential(
            Conv2d(3, 5, 11, 2),
            ReLU(),
            Conv2d(5, 10, 9, 2),
            ReLU(),
            Conv2d(10, 15, 3),
            ReLU(),
            View(-1, 50715),
            Linear(50715, 2048),
            ReLU(),
            Linear(2048, 256),
            ReLU(),
            Linear(256, 5)
        )

    def forward(self, x):
        return self._sequential(x)

In [ ]:
unit = device('cuda:0' if is_available() else 'cpu')

In [ ]:
auge = Auge().to(unit)
criterion = CrossEntropyLoss()
optimizer = Adam(auge.parameters())
training_losses, validation_losses = [], []
epoch_number = 6
for epoch in range(1, epoch_number + 1):
    start = time()
    training_loss, validation_loss = 0.0, 0.0
    auge.train()
    for samples, targets in training_loader:
        samples, targets = samples.to(unit), targets.to(unit)
        optimizer.zero_grad()
        loss = criterion(auge(samples), targets)
        loss.backward()
        optimizer.step()
        training_loss += loss.item() * samples.size(0)
    auge.eval()
    with no_grad():
        for samples, targets in validation_loader:
            samples, targets = samples.to(unit), targets.to(unit)
            validation_loss += criterion(auge(samples), targets).item() * samples.size(0)
    end = time()
    training_loss /= len(training_loader.sampler)
    validation_loss /= len(validation_loader.sampler)
    training_losses.append(training_loss)
    validation_losses.append(validation_loss)
    print(
        f'Epoch: {epoch:2}  '
        f'Training loss: {training_loss:8.6f}  '
        f'Validation loss: {validation_loss:8.6f}  '
        f'Duration: {end - start:.0f}s'
    )
save(auge.state_dict(), f'/content/drive/MyDrive/Scientific/V3/Models/auge.{uuid4().hex}.pth')
save(auge.state_dict(), f'/content/drive/MyDrive/Scientific/V3/Models/auge.latest.pth')
figure = Figure()
figure.add_trace(Scatter(x=arange(1, epoch_number + 1), y=training_losses, name='Training'))
figure.add_trace(Scatter(x=arange(1, epoch_number + 1), y=validation_losses, name='Validation'))
figure.update_layout(margin={'t': 30, 'r': 80, 'b': 0, 'l': 0})
figure.show()

In [ ]:
auge = Auge().to(unit)
auge.load_state_dict(load('/content/drive/MyDrive/Scientific/V3/Models/auge.latest.pth'))
auge.eval()
actual, expected = zeros(0, dtype=long).to(unit), zeros(0, dtype=long).to(unit)
with no_grad():
    for samples, targets in testing_loader:
        samples, targets = samples.to(unit), targets.to(unit)
        actual = cat([actual, max(auge(samples), 1)[1].view(-1)])
        expected = cat([expected, targets.view(-1)])
matrix = confusion_matrix(expected.cpu(), actual.cpu())
figure = create_annotated_heatmap(z=matrix, x=labels, y=labels, hoverinfo='skip')
figure.update_xaxes(title_text='Expected')
figure.update_yaxes(title_text='Actual', autorange='reversed')
figure.update_layout(
    margin={'t': 100, 'r': 0, 'b': 0, 'l': 0},
    title=f'Accuracy: {trace(matrix) / sum(matrix) * 100:.2f}%'
)
figure.show()

In [ ]:
# from cv2 import imread, IMREAD_COLOR, cvtColor, COLOR_BGR2RGB, IMREAD_GRAYSCALE, imwrite, inpaint, INPAINT_TELEA
# def d3(matrix):
#     return stack((matrix, matrix, matrix), 2)
# url = '../data/auge/images/d379f0f7ed01b7290a3a905e9daba1d1276ef7d3.comfort.webp'
# gray = imread(url, IMREAD_GRAYSCALE)
# mask = copy(gray)
# mask[341:443, 125:272] = 255
# mask[:341, :] = 0
# mask[443:, :] = 0
# mask[:, :125] = 0
# mask[:, 272:] = 0
# for x in range(125, 165):
#     mask[:382 + 126 - x, x] = 0
# for x in range(210, 272):
#     mask[:x + 130, x] = 0
# name, label = parse(url)
# iplot(
#     make_subplots(cols=2, horizontal_spacing=0.03)
#     .add_trace(Image(z=d3(gray), name=''), 1, 1)
#     .add_trace(Image(z=d3(mask), name=''), 1, 2)
#     .update_layout(title=f'{name} - {label}', margin={'t': 40, 'r': 0, 'b': 0, 'l': 0}, height=380)
#     .update_xaxes(showticklabels=False)
#     .update_yaxes(showticklabels=False)
# )
# imwrite('../data/auge/masks/olymp.jpeg', mask)
# url = '../data/auge/images/b7bb98d1f53cbc8614fad2e5926975a13904e135.construction.webp'
# bgr = imread(url, IMREAD_COLOR)
# name, label = parse(url)
# iplot(
#     make_subplots(cols=2, horizontal_spacing=0.03)
#     .add_trace(Image(z=cvtColor(bgr, COLOR_BGR2RGB), name=''), 1, 1)
#     .add_trace(
#         Image(
#             z=cvtColor(
#                 inpaint(bgr, imread('../data/auge/masks/domria.jpeg', IMREAD_GRAYSCALE), 4, INPAINT_NS),
#                 COLOR_BGR2RGB
#             ),
#             name=''
#         ),
#         1,
#         2
#     )
#     .update_layout(title=f'{name} - {label}', margin={'t': 40, 'r': 0, 'b': 0, 'l': 0}, height=380)
#     .update_xaxes(showticklabels=False)
#     .update_yaxes(showticklabels=False)
# )